<a href="https://colab.research.google.com/github/wiggitywalt/aap/blob/master/063_Zillow_For_Sale_Properties_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps
### Get Keys
1) Get Free [Rapid API Key by creating an account](https://rapidapi.com/marketplace)

Store each key in a central and private location.

### Subscribe to APIs

1) Subscribe to [Zillow on Rapid API](https://rapidapi.com/apimaker/api/zillow-com1/)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [ ]:
from google.colab import drive, files # specific to Google Colab
import pandas as pd
import requests
import json
import time

# show all columns
pd.set_option('display.max_columns', None)

## <font color="blue">Functions</font>

## <font color="blue">Local & Constants</font>

In [ ]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/' # optional

############
# REQURIED #
############
city = 'brandon'
state = 'fl'
search_str = city + ', ' + state
print('Search string:', search_str)

Mounted at /content/drive
Search string: brandon, fl


## <font color="blue">Data</font>

### Get API Keys

In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key

## <font color="green">Zillow</font>

### <font color="purple">Endpoint: Property</font>

In [ ]:
# get data
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_str,
               "home_type":"Houses",
               "minPrice": "250000",
               "maxPrice": "350000",
               "sqftMin": "1300"}

headers = {
    'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }

z_for_sale_resp = requests.request("GET", url, headers=headers, params=querystring)

# transform to json
z_for_sale_resp_json = z_for_sale_resp.json()
z_for_sale_resp_json

{'props': [{'address': '1806 Fruitridge St, Brandon, FL 33510',
   'bathrooms': 2,
   'bedrooms': 3,
   'country': 'USA',
   'currency': 'USD',
   'dateSold': None,
   'daysOnZillow': -1,
   'hasImage': True,
   'imgSrc': 'https://photos.zillowstatic.com/fp/2f71783b75f9c9aac1312379aa2cd475-p_e.jpg',
   'latitude': 27.967442,
   'listingDateTime': None,
   'listingStatus': 'FOR_SALE',
   'listingSubType': {'is_FSBA': True},
   'livingArea': 1397,
   'longitude': -82.310135,
   'lotAreaUnit': 'sqft',
   'lotAreaValue': 8276.4,
   'price': 325000,
   'propertyType': 'SINGLE_FAMILY',
   'zpid': '44969719'},
  {'address': '1006 Kenmore Dr, Brandon, FL 33510',
   'bathrooms': 1,
   'bedrooms': 3,
   'country': 'USA',
   'currency': 'USD',
   'dateSold': None,
   'daysOnZillow': -1,
   'hasImage': True,
   'imgSrc': 'https://photos.zillowstatic.com/fp/acdce9f89f63d3635e8c59197edbd52d-p_e.jpg',
   'latitude': 27.95466,
   'listingDateTime': None,
   'listingStatus': 'FOR_SALE',
   'listingSubT

In [ ]:
# view data
df_z_for_sale = pd.json_normalize(data=z_for_sale_resp_json['props'])
print('Num of rows:', len(df_z_for_sale))
print('Num of cols:', len(df_z_for_sale.columns))
df_z_for_sale.head()

Num of rows: 5
Num of cols: 20


,dateSold,propertyType,lotAreaValue,address,imgSrc,price,listingDateTime,longitude,latitude,listingStatus,zpid,livingArea,daysOnZillow,bedrooms,bathrooms,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA
0,None,SINGLE_FAMILY,8276.4,"1806 Fruitridge St, Brandon, FL 33510",https://photos.zillowstatic.com/fp/2f71783b75f...,325000,None,-82.310135,27.967442,FOR_SALE,44969719,1397,-1,3,2,USA,USD,sqft,True,True
1,None,SINGLE_FAMILY,9583.2,"1006 Kenmore Dr, Brandon, FL 33510",https://photos.zillowstatic.com/fp/acdce9f89f6...,315000,None,-82.291160,27.954660,FOR_SALE,44975729,1502,-1,3,1,USA,USD,sqft,True,True
2,None,SINGLE_FAMILY,7840.8,"1106 Briarwood Rd, Brandon, FL 33511",https://photos.zillowstatic.com/fp/7e455e6fc31...,335000,None,-82.266560,27.934002,FOR_SALE,44981568,1623,-1,5,2,USA,USD,sqft,True,True
3,None,SINGLE_FAMILY,7405.2,"811 Pebblewood Dr, Brandon, FL 33511",https://photos.zillowstatic.com/fp/7c88c8b2fd2...,325000,None,-82.295525,27.887197,FOR_SALE,44993981,1356,-1,4,2,USA,USD,sqft,True,True
4,None,SINGLE_FAMILY,9147.6,"117 Pauls Dr, Brandon, FL 33511",https://photos.zillowstatic.com/fp/dda746024f8...,349900,None,-82.301810,27.936316,FOR_SALE,44982609,1326,-1,3,1,USA,USD,sqft,True,True


In [ ]:
# download file


In [ ]:
# get zpids to a list
zpid_list = df_z_for_sale['zpid'].tolist()
zpid_list

['44969719', '44975729', '44981568', '44993981', '44982609']

In [ ]:
# get property detail

# create empty list
prop_detail_list = []

# iterate through list of properties
for zpid in zpid_list:

  # end point
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  # header
  headers = {
      'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
      'x-rapidapi-key': rapid_api_key
      }

  # get property detail
  z_prop_detail_resp = requests.request("GET", url, headers=headers, params=querystring)
  z_prop_detail_resp_json = z_prop_detail_resp.json()

  # wait 1 sec based on limit
  time.sleep(1.5)

  prop_detail_list.append(z_prop_detail_resp_json)

In [ ]:
# convert to dataframe
df_z_prop_detail = pd.json_normalize(prop_detail_list)
print('Num of rows:', len(df_z_prop_detail))
print('Num of cols:', len(df_z_prop_detail.columns))
df_z_prop_detail.head(2)

Num of rows: 5
Num of cols: 259


,listingProvider,buildingPermits,propertyTaxRate,contact_recipients,longitude,countyFIPS,cityId,timeOnZillow,url,zestimate,imgSrc,zpid,livingAreaValue,zipcode,zestimateLowPercent,brokerId,streetAddress,county,stateId,taxHistory,countyId,timeZone,homeType,livingAreaUnits,pageViewCount,livingArea,bathrooms,annualHomeownersInsurance,state,building,rentZestimate,yearBuilt,brokerageName,propertyTypeDimension,isListedByOwner,price,description,homeStatus,homeFacts,latitude,datePosted,bedrooms,nearbyHomes,monthlyHoaFee,priceHistory,favoriteCount,schools,zestimateHighPercent,mlsid,dateSold,city,providerListingID,country,currency,contingentListingType,solarPotential.sunScore,solarPotential.solarFactor,solarPotential.buildFactor,solarPotential.climateFactor,solarPotential.electricityFactor,address.city,address.neighborhood,address.state,address.streetAddress,address.zipcode,resoFacts.hasAttachedProperty,resoFacts.frontageType,resoFacts.poolFeatures,resoFacts.flooring,resoFacts.foundationDetails,resoFacts.accessibilityFeatures,resoFacts.hasGarage,resoFacts.hasPetsAllowed,resoFacts.bodyType,resoFacts.topography,resoFacts.landLeaseExpirationDate,resoFacts.hasAdditionalParcels,resoFacts.waterViewYN,resoFacts.atAGlanceFacts,resoFacts.offerReviewDate,resoFacts.aboveGradeFinishedArea,resoFacts.buyerAgencyCompensationType,resoFacts.carportSpaces,resoFacts.cityRegion,resoFacts.mainLevelBathrooms,resoFacts.hasPrivatePool,resoFacts.associationFeeIncludes,resoFacts.waterSource,resoFacts.exteriorFeatures,resoFacts.coveredSpaces,resoFacts.inclusions,resoFacts.hasFireplace,resoFacts.zoningDescription,resoFacts.garageSpaces,resoFacts.sewer,resoFacts.elevationUnits,resoFacts.exclusions,resoFacts.mainLevelBedrooms,resoFacts.numberOfUnitsVacant,resoFacts.hasWaterfrontView,resoFacts.listAOR,resoFacts.lotSize,resoFacts.entryLevel,resoFacts.irrigationWaterRightsAcres,resoFacts.greenWaterConservation,resoFacts.stories,resoFacts.livingArea,resoFacts.numberOfUnitsInCommunity,resoFacts.listingTerms,resoFacts.otherStructures,resoFacts.associationFee,resoFacts.marketingType,resoFacts.greenIndoorAirQuality,resoFacts.greenSustainability,resoFacts.hasHomeWarranty,resoFacts.associationPhone,resoFacts.greenBuildingVerificationType,resoFacts.elementarySchool,resoFacts.bedrooms,resoFacts.architecturalStyle,resoFacts.listingId,resoFacts.structureType,resoFacts.interiorFeatures,resoFacts.horseAmenities,resoFacts.electric,resoFacts.developmentStatus,resoFacts.lotFeatures,resoFacts.roofType,resoFacts.waterSources,resoFacts.constructionMaterials,resoFacts.fireplaceFeatures,resoFacts.doorFeatures,resoFacts.appliances,resoFacts.daysOnZillow,resoFacts.bathroomsPartial,resoFacts.bathroomsHalf,resoFacts.depositsAndFees,resoFacts.canRaiseHorses,resoFacts.buildingName,resoFacts.hasAssociation,resoFacts.petsMaxWeight,resoFacts.commonWalls,resoFacts.totalActualRent,resoFacts.additionalParcelsDescription,resoFacts.associationName,resoFacts.allowedPets,resoFacts.communityFeatures,resoFacts.tenantPays,resoFacts.middleOrJuniorSchool,resoFacts.lotSizeDimensions,resoFacts.roadSurfaceType,resoFacts.hasOpenParking,resoFacts.yearBuilt,resoFacts.livingAreaRangeUnits,resoFacts.irrigationWaterRightsYN,resoFacts.bathroomsFull,resoFacts.horseYN,resoFacts.incomeIncludes,resoFacts.highSchool,resoFacts.leaseTerm,resoFacts.elevation,resoFacts.cropsIncludedYN,resoFacts.attic,resoFacts.subdivisionName,resoFacts.windowFeatures,resoFacts.ownership,resoFacts.woodedArea,resoFacts.middleOrJuniorSchoolDistrict,resoFacts.associationPhone2,resoFacts.buyerAgencyCompensation,resoFacts.heating,resoFacts.frontageLength,resoFacts.virtualTourURLUnbranded,resoFacts.associationAmenities,resoFacts.propertySubType,resoFacts.hasCooling,resoFacts.belowGradeFinishedArea,resoFacts.hasElectricOnProperty,resoFacts.foundationArea,resoFacts.zoning,resoFacts.hoaFee,resoFacts.livingAreaRange,resoFacts.basementYN,resoFacts.availabilityDate,resoFacts.waterBodyName,resoFacts.waterfrontFeatures,resoFacts.specialListingConditions,resoFac

In [ ]:
# columns of interest
detail_cols = ['streetAddress', 
 'city',
 'county',
 'zipcode',
 'state',
 'price',
 'homeType',
 'timeOnZillow', 
 'zestimate',
 'rentZestimate',
 'livingArea',
 'bedrooms',
 'bathrooms',
 'yearBuilt',
 'description',
 'priceHistory',
 'taxHistory',
 'zpid'
 ]

# retain limited columns for output
df_z_prop_detail_output = df_z_prop_detail[detail_cols]
df_z_prop_detail_output.head()

,streetAddress,city,county,zipcode,state,price,homeType,timeOnZillow,zestimate,rentZestimate,livingArea,bedrooms,bathrooms,yearBuilt,description,priceHistory,taxHistory,zpid
0,1806 Fruitridge St,Brandon,None,33510,FL,325000,SINGLE_FAMILY,2 days,NaN,NaN,1397,3,2,1986,"** A HOME OF LOVE, JOY, LAUGHTER & COMFORT **P...","[{'priceChangeRate': 1.9545454545455, 'date': ...","[{'time': 1582066675669, 'valueIncreaseRate': ...",44969719
1,1006 Kenmore Dr,Brandon,None,33510,FL,315000,SINGLE_FAMILY,141 days,314700.0,1995.0,1502,3,1,1961,One or more photo(s) has been virtually staged...,"[{'priceChangeRate': 0, 'date': '2022-02-16', ...","[{'time': 1582066678520, 'valueIncreaseRate': ...",44975729
2,1106 Briarwood Rd,Brandon,None,33511,FL,335000,SINGLE_FAMILY,98 days,335100.0,1909.0,1623,5,2,1959,Check out this newly remodeled home ready for ...,"[{'priceChangeRate': 0, 'date': '2022-02-11', ...","[{'time': 1582066681154, 'valueIncreaseRate': ...",44981568
3,811 Pebblewood Dr,Brandon,None,33511,FL,325000,SINGLE_FAMILY,15 days,338100.0,2173.0,1356,4,2,1979,One or more photo(s) has been virtually staged...,"[{'priceChangeRate': 0, 'date': '2022-02-16', ...","[{'time': 1582066683899, 'valueIncreaseRate': ...",44993981
4,117 Pauls Dr,Brandon,None,33511,FL,349900,SINGLE_FAMILY,34 days,349900.0,1837.0,1326,3,1,1968,Part of the Brandon Main Street Zoning. This ...,"[{'priceChangeRate': 0.75829145728643, 'date':...","[{'time': 1582066686578, 'valueIncreaseRate': ...",44982609


In [ ]:
# download file
df_z_prop_detail_output.to_csv('df_z_prop_detail_output.csv')
files.download('df_z_prop_detail_output.csv')

## Zillow ZPID
Useful to retrieve zillow detailed data for ANY address

In [ ]:
!pip install python-zillow -q

In [ ]:
# list of addresses
address_dict = {
    'address': [
                '125 Lithia Pinecrest Rd, Brandon, FL',
                '705 Westwood Ln, Brandon, FL',
                '101 Ruby Cir, Brandon, FL '], 
    'postal_code': [
                    33511, 
                    33511, 
                    33510]
}
df_addresses = pd.DataFrame(data=address_dict)
df_addresses

,address,postal_code
0,"125 Lithia Pinecrest Rd, Brandon, FL",33511
1,"705 Westwood Ln, Brandon, FL",33511
2,"101 Ruby Cir, Brandon, FL",33510


In [ ]:
zillow_api_key = df_api_keys.loc[df_api_keys['API'] =='zillow']['KEY'].iloc[0] # replace this with your own key

In [ ]:
import zillow

# create api instance
zillow_api = zillow.ValuationApi()

# create empty list
zpid_list = []

# iterate through dataframe
for index, row in df_addresses.iterrows():
  # get values to pass through api
  _address = row['address']
  _postal_code = row['postal_code']
  print(_address, _postal_code)

  # get search results
  _search_results = zillow_api.GetSearchResults(zillow_api_key, _address, _postal_code)

  # append zpid
  zpid_list.append(_search_results.zpid)

125 Lithia Pinecrest Rd, Brandon, FL 33511


ExpatError: ignored

In [ ]:
df_addresses['zpid'] = zpid_list
df_addresses

In [ ]:
# get property detail

# create empty list
prop_detail_list = []

# iterate through list of properties
for zpid in zpid_list:

  # end point
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  # header
  headers = {
      'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
      'x-rapidapi-key': rapid_api_key
      }

  # get property detail
  z_prop_detail_resp = requests.request("GET", url, headers=headers, params=querystring)
  z_prop_detail_resp_json = z_prop_detail_resp.json()

  # wait 1 sec based on limit
  time.sleep(1.5)

  prop_detail_list.append(z_prop_detail_resp_json)

In [ ]:
# convert to dataframe
df_z_prop_detail = pd.json_normalize(prop_detail_list)
print('Num of rows:', len(df_z_prop_detail))
print('Num of cols:', len(df_z_prop_detail.columns))
df_z_prop_detail.head(2)

In [ ]:
# columns of interest
detail_cols = ['streetAddress', 
 'city',
 'county',
 'zipcode',
 'state',
 'price',
 'homeType',
 'timeOnZillow', 
 'zestimate',
 'rentZestimate',
 'livingArea',
 'bedrooms',
 'bathrooms',
 'yearBuilt',
 'description',
 'priceHistory',
 'taxHistory',
 'zpid'
 ]

# retain limited columns for output
df_z_prop_detail_output = df_z_prop_detail[detail_cols]
df_z_prop_detail_output.head()

# End Notebook